In [ ]:
import wandb

# Wandb API for CI/CD

The following are examples of operations you can perform with the `wandb` python client that might be relevant for CI/CD.

## Getting metrics tied to the current production model

We can get the production model from the registry, then get the metrics via lineage from the associated run.

In [ ]:
run = wandb.init(job_type="CI/CD")
path = 'av-team/model-registry/'
model_name = 'BDD Semantic Segmentation'
version = 'production'

artifact = run.use_artifact(f'{path}{model_name}:{version}', 
                            type='model')

wandb: Currently logged in as: hamelsmu (av-team). Use `wandb login --relogin` to force relogin


Get the run that created the model:

In [ ]:
run = artifact.logged_by()

In [ ]:
def get_iou_metrics(run):
    return {k:v for k,v in run.summary.items() if '_iou' in k}

get_iou_metrics(run)

{'bicycle_iou': 0,
 'vehicle_iou': 0.7598675070628712,
 'person_iou': 0.13954558401437303,
 'traffic_sign_iou': 0.04609753307660045,
 'final_traffic_sign_iou': 0.05314886217585242,
 'background_iou': 0.921853196957014,
 'final_road_iou': 0.8347413133027304,
 'final_person_iou': 0.20271520471610835,
 'final_background_iou': 0.920883500388814,
 'final_vehicle_iou': 0.7561426671932289,
 'traffic_light_iou': 0.1546196566812521,
 'road_iou': 0.8379052798838932,
 'final_traffic_light_iou': 0.11524698900331648,
 'final_bicycle_iou': 0}

## Getting metrics for a run ID

This is useful if you want to get metrics for a particular experiment ID.

In [ ]:
run_path = 'av-team/mlops-course-001/wlt1r8k7'
api = wandb.Api()
run = api.run(run_path)

In [ ]:
get_iou_metrics(run)

{'final_person_iou': 0,
 'final_bicycle_iou': 0,
 'final_traffic_light_iou': 0.09025893958076447,
 'person_iou': 0,
 'final_background_iou': 0.9194038173285132,
 'bicycle_iou': 0,
 'final_vehicle_iou': 0.7519664371203332,
 'vehicle_iou': 0.7519784886198334,
 'final_road_iou': 0.8422501240160545,
 'road_iou': 0.8467596086120717,
 'background_iou': 0.9209043224804184,
 'final_traffic_sign_iou': 0.00013421917992081068,
 'traffic_sign_iou': 0.0001342732460557234,
 'traffic_light_iou': 0.07797093928386092}

## Getting metrics for a run with a tag

It's better to use the [Mongo API](https://docs.wandb.ai/guides/track/public-api-guide#querying-multiple-runs) instead of downloading and iterating through each run if possible.

The use case for this is if you wanted to compare a particular run with another run that you have tagged, such as "baseline".

In [ ]:
api = wandb.Api()
tags= ['candidate']

baseline_runs=api.runs('av-team/mlops-course-001', 
                       {"tags": {"$in": tags}}) # this is the Mongo Client

In [ ]:
for run in baseline_runs:
    print(run.name)

scarlet-armadillo-502
fine-sweep-63


## Promote a model to the registry

This is also might be something you could do as part of a CI/CD process.

In [ ]:
path = 'av-team/model-registry/BDD Semantic Segmentation'

In [ ]:
run_path = 'av-team/mlops-course-001/wlt1r8k7' # this is the run id
api = wandb.Api()
run = api.run(run_path)

Get the model from the run and promote it to the registry

In [ ]:
art = [a for a in run.logged_artifacts() if a.type == 'model']

if art:
    assert len(art) == 1, 'More then 1 artifact of type model!'
    art[0].link(path, aliases=['prod candidate'])

We can see a model in the registry like this:

In [ ]:
api = wandb.Api()
versions = api.artifact_versions('model', path)

In [ ]:
for v in versions:
    print(f'Model version: {v.version} with tags: {v.aliases}')

Model version: v2 with tags: ['latest', 'prod candidate']
Model version: v1 with tags: ['staging', 'production']
Model version: v0 with tags: []
